# Practical case 1 - Vehicle consumption 

In [1]:
%pylab
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

import numpy as np

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In the `data/auto.csv` file there are the following data of different vehicles:

- cylinders
- displacement
- horsepower
- weight
- acceleration
- model_year
- origin
- mpg

The units of the features are not in the IS. The `origin` variable is the origin country code of the vehicle.

You have to make a model with that dataset which estimates the vehicle consumption (`mpg` variable) according to the others variables

### Load data

In [2]:
import pandas as pd

# Read file
auto = pd.read_csv("data/auto.csv", sep = ",")
auto.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg
0,8,307.0,130.0,3504.0,12.0,70,1,18.0
1,8,350.0,165.0,3693.0,11.5,70,1,15.0
2,8,318.0,150.0,3436.0,11.0,70,1,18.0
3,8,304.0,150.0,3433.0,12.0,70,1,16.0
4,8,302.0,140.0,3449.0,10.5,70,1,17.0


In [3]:
# Remove the 'mpg' variable of features
target_name = 'mpg'
features_names = list(auto.columns)
features_names.remove(target_name)

# Split the features and target data
features = auto[features_names]
target = auto[target_name]

### VIF - Variance Inflation Factor

The multicollinearity in a independient features set impacts negatively to the models built with them. A solution of this problem is use the VIF which allows cuantify the intensity of multicollinearity. The VIF value increases according to the multicollinearity increases. The VIF values bigger than 5 are considered high and VIF values bigger than 10 are considered very high.

The VIF value calculation can be implement like this:

In [4]:
def calculateVIF(data) :
    from sklearn.linear_model import LinearRegression
    import pandas as pd
    
    features = list(data.columns)
    num_features = len(features)
    
    # Create the model and the result dataframe
    model = LinearRegression()
    result = pd.DataFrame(index = ['VIF'], columns = features)
    result = result.fillna(0)
    
    # For each feature
    for ite in range(num_features) :
        x_features = features[:]
        y_feature  = features[ite]
        # Remove the feature (because it is the independient)
        x_features.remove(y_feature)
        
        x = data[x_features]
        y = data[y_feature]
        
        # Fit the model 
        model.fit(x, y)
        # Calculate VIF
        result[y_feature] = 1 / (1 - model.score(x, y))
    
    return result

So, we are going to calculate the VIF value for each feature in our dataset: 

In [5]:
VIF = calculateVIF(features)
VIF

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
VIF,10.737535,21.836792,9.943693,10.83126,2.625806,1.244952,1.772386


Once VIF method has been defined, a procedure which remove the features is needed. This procedure is the following: 

In [6]:
# data    -> data
# max_VIF -> maximum VIF value to follow removing features
def selectDataUsingVIF(data, max_VIF = 5) :
    # Copy data
    result = data.copy(deep = True)
    
    VIF = calculateVIF(result)
    
    # While the VIF value is bigger than max_VIF:
    while VIF.as_matrix().max() > max_VIF :
        # Get the column of the feature which gets the maximum VIF
        col_max = np.where(VIF == VIF.as_matrix().max())[1][0]
        
        # Remove this feature of the data
        features = list(result.columns)
        features.remove(features[col_max])
        result = result[features]
        
        # Again, calculate VIF
        VIF = calculateVIF(result)

    # Return the result
    return result

Now, we are going to select data using VIF:

In [7]:
selected_features = selectDataUsingVIF(features)
calculateVIF(selected_features)

,cylinders,acceleration,model_year,origin
VIF,1.999959,1.384478,1.159429,1.495041


Once we have removed the features with multicollinearity, we are going to make a model with selected features. That model will be a `Linear Regression`. For training the model, we are going to use crossvalidation for the model will not be overfitting.

Let's implement:

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn.model_selection import train_test_split

In [9]:
# Default = 75% train, 25% test
x_train, x_test, y_train, y_test = train_test_split(selected_features, target,
                                                    # train_size, for change default values
                                                    # test_size, for change default values
                                                    random_state = 1) # for the split don't change in each execution

# Create a model and fit it with de train data
model = LinearRegression()
model.fit(x_train, y_train)

# Let's predict the x_test data
pred = model.predict(x_test)

print("The model's metrics are:")
print('Trainning R^2 = ', model.score(x_train, y_train))
print('Testing R^2 = ', model.score(x_test, y_test))
print('MSE = ', mean_squared_error(pred, y_test))
print('MAE = ', mean_absolute_error(pred, y_test))
print('MedianAE = ', median_absolute_error(pred, y_test))

The model's metrics are:
Trainning R^2 =  0.723846058505
Testing R^2 =  0.763162966762
MSE =  16.8929209872
MAE =  3.00502160024
MedianAE =  2.22425381474


With that, we can say the model is not overfitting and the metrics are good. So, we have made the model which estimates the vehicle consumption.

For curiosity, we are going to make that model with all features. Like this, we see if the model with all features is better than our model:

In [10]:
# Default = 75% train, 25% test
x_train, x_test, y_train, y_test = train_test_split(features, target,
                                                    # train_size, for change default values
                                                    # test_size, for change default values
                                                    random_state = 1) # for the split don't change in each execution

# Create a model and fit it with de train data
all_features_model = LinearRegression()
all_features_model.fit(x_train, y_train)

# Let's predict the x_test data
pred = all_features_model.predict(x_test)

print("The model's metrics are:")
print('Trainning R^2 = ', all_features_model.score(x_train, y_train))
print('Testing R^2 = ', all_features_model.score(x_test, y_test))
print('MSE = ', mean_squared_error(pred, y_test))
print('MAE = ', mean_absolute_error(pred, y_test))
print('MedianAE = ', median_absolute_error(pred, y_test))

The model's metrics are:
Trainning R^2 =  0.815333962764
Testing R^2 =  0.829570053694
MSE =  12.1562898227
MAE =  2.63899727715
MedianAE =  2.13787537813


As we can observe, the metrics are a little better, but that model is more complex because it use more features and them have multicollinearity, so we have made a model more simple and without features multicollinearity. 